In [0]:
[
    {'user_id': 1, 'user_fname': 'Scott', 'user_lname': 'Tiger'},
    {'user_id': 2, 'user_fname': 'Donald', 'user_lname': 'Duck'},
    {'user_id': 3, 'user_fname': 'Mickey', 'user_lname': 'Mouse'}
]

[{'user_id': 1, 'user_fname': 'Scott', 'user_lname': 'Tiger'},
 {'user_id': 2, 'user_fname': 'Donald', 'user_lname': 'Duck'},
 {'user_id': 3, 'user_fname': 'Mickey', 'user_lname': 'Mouse'}]

In [0]:
orders_df = spark.read.csv('dbfs:/public/retail_db/orders/part-00000', schema='order_id INT, order_date date, order_customer_id int, order_status string')

In [0]:
display(orders_df.limit(10))

order_id,order_date,order_customer_id,order_status
1,2013-07-25,11599,CLOSED
2,2013-07-25,256,PENDING_PAYMENT
3,2013-07-25,12111,COMPLETE
4,2013-07-25,8827,CLOSED
5,2013-07-25,11318,COMPLETE
6,2013-07-25,7130,COMPLETE
7,2013-07-25,4530,COMPLETE
8,2013-07-25,2911,PROCESSING
9,2013-07-25,5657,PENDING_PAYMENT
10,2013-07-25,5648,PENDING_PAYMENT


In [0]:
from pyspark.sql.functions import date_format, cast

In [0]:
display(orders_df.withColumn('order_month', cast('int', date_format('order_date', 'yyyyMM'))).limit(10))

order_id,order_date,order_customer_id,order_status,order_month
1,2013-07-25,11599,CLOSED,201307
2,2013-07-25,256,PENDING_PAYMENT,201307
3,2013-07-25,12111,COMPLETE,201307
4,2013-07-25,8827,CLOSED,201307
5,2013-07-25,11318,COMPLETE,201307
6,2013-07-25,7130,COMPLETE,201307
7,2013-07-25,4530,COMPLETE,201307
8,2013-07-25,2911,PROCESSING,201307
9,2013-07-25,5657,PENDING_PAYMENT,201307
10,2013-07-25,5648,PENDING_PAYMENT,201307


In [0]:
orders_df.withColumn('order_month', date_format('order_date', 'yyyyMM').cast('int')).printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- order_customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_month: integer (nullable = true)



In [0]:
display(orders_df.filter("order_status = 'COMPLETE'").limit(10))

order_id,order_date,order_customer_id,order_status
3,2013-07-25,12111,COMPLETE
5,2013-07-25,11318,COMPLETE
6,2013-07-25,7130,COMPLETE
7,2013-07-25,4530,COMPLETE
15,2013-07-25,2568,COMPLETE
17,2013-07-25,2667,COMPLETE
22,2013-07-25,333,COMPLETE
26,2013-07-25,7562,COMPLETE
28,2013-07-25,656,COMPLETE
32,2013-07-25,3960,COMPLETE


In [0]:
orders_df.filter("order_status = 'COMPLETE'").count()

22899

get all the COMPLETE or CLOSED orders which are palced in the month of January of 2014

In [0]:
display(orders_df.filter("order_status in ('COMPLETE', 'CLOSED')  and year(order_date) = 2014 and month(order_date) = 1").limit(10))

order_id,order_date,order_customer_id,order_status
25882,2014-01-01,4598,COMPLETE
25888,2014-01-01,6735,COMPLETE
25889,2014-01-01,10045,COMPLETE
25891,2014-01-01,3037,CLOSED
25895,2014-01-01,1044,COMPLETE
25897,2014-01-01,6405,COMPLETE
25898,2014-01-01,3950,COMPLETE
25899,2014-01-01,8068,CLOSED
25900,2014-01-01,2382,CLOSED
25901,2014-01-01,3099,COMPLETE


In [0]:
from pyspark.sql.functions import count, col

In [0]:
display(orders_df. \
    groupBy('order_status'). \
    agg(count('order_id').alias('order_count')). \
    orderBy(col('order_count').desc()))

order_status,order_count
COMPLETE,22899
PENDING_PAYMENT,15030
PROCESSING,8275
PENDING,7610
CLOSED,7556
ON_HOLD,3798
SUSPECTED_FRAUD,1558
CANCELED,1428
PAYMENT_REVIEW,729


In [0]:
order_items_df = spark.read.csv('dbfs:/public/retail_db/order_items/part-00000', schema='order_id int, order_item_order_id int, order_item_product_id int, order_item_quantity int, order_item_subtotal float, order_item_product_price float')

In [0]:
display(order_items_df.limit(10))

order_id,order_item_order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price
1,1,957,1,299.98,299.98
2,2,1073,1,199.99,199.99
3,2,502,5,250.0,50.0
4,2,403,1,129.99,129.99
5,4,897,2,49.98,24.99
6,4,365,5,299.95,59.99
7,4,502,3,150.0,50.0
8,4,1014,4,199.92,49.98
9,5,957,1,299.98,299.98
10,5,365,5,299.95,59.99


In [0]:
from pyspark.sql.functions import sum, round

In [0]:
display(order_items_df.groupBy('order_item_order_id'). \
    agg(round(sum('order_item_subtotal'), 2).alias('order_revenue')). \
    orderBy('order_item_order_id').limit(10)
)

order_item_order_id,order_revenue
1,299.98
2,579.98
4,699.85
5,1129.86
7,579.92
8,729.84
9,599.96
10,651.92
11,919.79
12,1299.87


get the count of orders per month

In [0]:
from pyspark.sql.functions import year, month

In [0]:
display(orders_df.groupBy(year('order_date').alias('year'), month('order_date').alias('month')). \
    agg(count('order_id')).alias('order_count'). \
    orderBy('year', 'month'))

year,month,count(order_id)
2013,7,1533
2013,8,5680
2013,9,5841
2013,10,5335
2013,11,6381
2013,12,5892
2014,1,5908
2014,2,5635
2014,3,5778
2014,4,5657


In [0]:
display(orders_df.orderBy('order_customer_id').limit(10))

order_id,order_date,order_customer_id,order_status
22945,2013-12-13,1,COMPLETE
15192,2013-10-29,2,PENDING_PAYMENT
33865,2014-02-18,2,COMPLETE
57963,2013-08-02,2,ON_HOLD
67863,2013-11-30,2,COMPLETE
35158,2014-02-26,3,COMPLETE
22646,2013-12-11,3,COMPLETE
23662,2013-12-19,3,COMPLETE
46399,2014-05-09,3,PROCESSING
56178,2014-07-15,3,PENDING


In [0]:
display(orders_df.orderBy(col('order_customer_id').desc()).limit(10))

order_id,order_date,order_customer_id,order_status
41643,2014-04-08,12435,PENDING
61629,2013-12-21,12435,CANCELED
1868,2013-08-03,12434,CLOSED
4799,2013-08-23,12434,PENDING_PAYMENT
5303,2013-08-26,12434,PENDING
6160,2013-09-02,12434,COMPLETE
13544,2013-10-16,12434,PENDING
42915,2014-04-16,12434,COMPLETE
51800,2014-06-14,12434,ON_HOLD
61777,2013-12-26,12434,COMPLETE


sort the data in ascending order by customer_id and order_date

In [0]:
display(orders_df.orderBy(col('order_customer_id'), col('order_date')).limit(10))

order_id,order_date,order_customer_id,order_status
22945,2013-12-13,1,COMPLETE
57963,2013-08-02,2,ON_HOLD
15192,2013-10-29,2,PENDING_PAYMENT
67863,2013-11-30,2,COMPLETE
33865,2014-02-18,2,COMPLETE
22646,2013-12-11,3,COMPLETE
61453,2013-12-14,3,COMPLETE
23662,2013-12-19,3,COMPLETE
35158,2014-02-26,3,COMPLETE
46399,2014-05-09,3,PROCESSING


In [0]:
display(order_items_df.orderBy(col('order_item_order_id'), col('order_item_subtotal').desc()).limit(10))

order_id,order_item_order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price
1,1,957,1,299.98,299.98
3,2,502,5,250.0,50.0
2,2,1073,1,199.99,199.99
4,2,403,1,129.99,129.99
6,4,365,5,299.95,59.99
8,4,1014,4,199.92,49.98
7,4,502,3,150.0,50.0
5,4,897,2,49.98,24.99
9,5,957,1,299.98,299.98
12,5,957,1,299.98,299.98


In [0]:
%fs ls dbfs:/databricks-datasets

path,name,size,modificationTime
dbfs:/databricks-datasets/COVID/,COVID/,0,1696018064038
dbfs:/databricks-datasets/README.md,README.md,976,1532468253000
dbfs:/databricks-datasets/Rdatasets/,Rdatasets/,0,1696018064038
dbfs:/databricks-datasets/SPARK_README.md,SPARK_README.md,3359,1455043490000
dbfs:/databricks-datasets/adult/,adult/,0,1696018064038
dbfs:/databricks-datasets/airlines/,airlines/,0,1696018064038
dbfs:/databricks-datasets/amazon/,amazon/,0,1696018064038
dbfs:/databricks-datasets/asa/,asa/,0,1696018064038
dbfs:/databricks-datasets/atlas_higgs/,atlas_higgs/,0,1696018064038
dbfs:/databricks-datasets/bikeSharing/,bikeSharing/,0,1696018064038


In [0]:
%fs ls dbfs:/databricks-datasets/online_retail/data-001/

path,name,size,modificationTime
dbfs:/databricks-datasets/online_retail/data-001/data.csv,data.csv,5357240,1466107812000


In [0]:
df = spark.read.csv('dbfs:/databricks-datasets/online_retail/data-001/', header=True, inferSchema=True)

In [0]:
display(df.limit(10))

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850,United Kingdom
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850,United Kingdom
536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/10 8:26,7.65,17850,United Kingdom
536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/10 8:26,4.25,17850,United Kingdom
536366,22633,HAND WARMER UNION JACK,6,12/1/10 8:28,1.85,17850,United Kingdom
536366,22632,HAND WARMER RED POLKA DOT,6,12/1/10 8:28,1.85,17850,United Kingdom
536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/10 8:34,1.69,13047,United Kingdom


In [0]:
df.count()

65499

In [0]:
display(df.filter("customerid is null").limit(10))

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536414,22139,null,56,12/1/10 11:52,0.0,null,United Kingdom
536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/10 14:32,2.51,null,United Kingdom
536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,12/1/10 14:32,2.51,null,United Kingdom
536544,21786,POLKADOT RAIN HAT,4,12/1/10 14:32,0.85,null,United Kingdom
536544,21787,RAIN PONCHO RETROSPOT,2,12/1/10 14:32,1.66,null,United Kingdom
536544,21790,VINTAGE SNAP CARDS,9,12/1/10 14:32,1.66,null,United Kingdom
536544,21791,VINTAGE HEADS AND TAILS CARD GAME,2,12/1/10 14:32,2.51,null,United Kingdom
536544,21801,CHRISTMAS TREE DECORATION WITH BELL,10,12/1/10 14:32,0.43,null,United Kingdom
536544,21802,CHRISTMAS TREE HEART DECORATION,9,12/1/10 14:32,0.43,null,United Kingdom
536544,21803,CHRISTMAS TREE STAR DECORATION,11,12/1/10 14:32,0.43,null,United Kingdom


In [0]:
display(df.filter("stockcode = 71053").limit(10))

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850,United Kingdom
536373,71053,WHITE METAL LANTERN,6,12/1/10 9:02,3.39,17850,United Kingdom
536375,71053,WHITE METAL LANTERN,6,12/1/10 9:32,3.39,17850,United Kingdom
536396,71053,WHITE METAL LANTERN,6,12/1/10 10:51,3.39,17850,United Kingdom
536406,71053,WHITE METAL LANTERN,8,12/1/10 11:33,3.39,17850,United Kingdom
536544,71053,WHITE METAL LANTERN,1,12/1/10 14:32,8.47,null,United Kingdom
536600,71053,WHITE METAL LANTERN,6,12/2/10 8:32,3.39,17850,United Kingdom
536602,71053,WHITE METAL LANTERN,6,12/2/10 8:34,3.39,17850,United Kingdom
536609,71053,WHITE METAL LANTERN,6,12/2/10 9:41,3.39,17850,United Kingdom
536612,71053,WHITE METAL LANTERN,6,12/2/10 9:44,3.39,17850,United Kingdom


In [0]:
display(df.filter("stockcode = 71053"). \
    orderBy('customerid'). \
    limit(10)
)

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536865,71053,WHITE METAL LANTERN,1,12/3/10 11:28,8.47,null,United Kingdom
536544,71053,WHITE METAL LANTERN,1,12/1/10 14:32,8.47,null,United Kingdom
537237,71053,WHITE METAL LANTERN,1,12/6/10 9:58,8.47,null,United Kingdom
537240,71053,WHITE METAL LANTERN,1,12/6/10 10:08,8.47,null,United Kingdom
537434,71053,WHITE METAL LANTERN,2,12/6/10 16:57,8.47,null,United Kingdom
537642,71053,WHITE METAL LANTERN,1,12/7/10 15:33,8.47,null,United Kingdom
539737,71053,WHITE METAL LANTERN,1,12/21/10 15:19,8.47,null,United Kingdom
539437,71053,WHITE METAL LANTERN,1,12/17/10 14:54,8.47,null,United Kingdom
539453,71053,WHITE METAL LANTERN,1,12/17/10 17:08,8.47,null,United Kingdom
539736,71053,WHITE METAL LANTERN,1,12/21/10 15:18,8.47,null,United Kingdom


In [0]:
from pyspark.sql.functions import asc_nulls_last

In [0]:
display(df.filter("stockcode = 71053"). \
    orderBy(col('CustomerId').asc_nulls_last()). \
    limit(10)
)

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
537759,71053,WHITE METAL LANTERN,4,12/8/10 11:42,3.75,13126,United Kingdom
537387,71053,WHITE METAL LANTERN,4,12/6/10 13:34,3.75,13468,United Kingdom
537649,71053,WHITE METAL LANTERN,4,12/7/10 15:48,3.75,13807,United Kingdom
540464,71053,WHITE METAL LANTERN,8,1/7/11 13:17,3.39,14298,United Kingdom
538012,71053,WHITE METAL LANTERN,3,12/9/10 12:32,3.75,14527,United Kingdom
539002,71053,WHITE METAL LANTERN,1,12/15/10 12:30,3.75,14527,United Kingdom
539326,71053,WHITE METAL LANTERN,4,12/17/10 9:22,3.75,14560,United Kingdom
540955,71053,WHITE METAL LANTERN,2,1/12/11 13:27,3.75,14715,United Kingdom
540493,71053,WHITE METAL LANTERN,4,1/7/11 16:34,3.75,14867,United Kingdom
540184,71053,WHITE METAL LANTERN,4,1/5/11 13:40,3.75,15266,United Kingdom


In [0]:
display(df.filter("stockcode = 71053"). \
    orderBy(col('CustomerId').desc_nulls_first()). \
    limit(10)
)

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536544,71053,WHITE METAL LANTERN,1,12/1/10 14:32,8.47,null,United Kingdom
539736,71053,WHITE METAL LANTERN,1,12/21/10 15:18,8.47,null,United Kingdom
536865,71053,WHITE METAL LANTERN,1,12/3/10 11:28,8.47,null,United Kingdom
537237,71053,WHITE METAL LANTERN,1,12/6/10 9:58,8.47,null,United Kingdom
537240,71053,WHITE METAL LANTERN,1,12/6/10 10:08,8.47,null,United Kingdom
537434,71053,WHITE METAL LANTERN,2,12/6/10 16:57,8.47,null,United Kingdom
537642,71053,WHITE METAL LANTERN,1,12/7/10 15:33,8.47,null,United Kingdom
539437,71053,WHITE METAL LANTERN,1,12/17/10 14:54,8.47,null,United Kingdom
539453,71053,WHITE METAL LANTERN,1,12/17/10 17:08,8.47,null,United Kingdom
539737,71053,WHITE METAL LANTERN,1,12/21/10 15:19,8.47,null,United Kingdom


In [0]:
order_details_df = orders_df.join(order_items_df, orders_df['order_id'] == order_items_df['order_item_order_id'])

In [0]:
display(order_details_df.limit(10))

order_id,order_date,order_customer_id,order_status,order_id,order_item_order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price
1,2013-07-25,11599,CLOSED,1,1,957,1,299.98,299.98
2,2013-07-25,256,PENDING_PAYMENT,2,2,1073,1,199.99,199.99
2,2013-07-25,256,PENDING_PAYMENT,3,2,502,5,250.0,50.0
2,2013-07-25,256,PENDING_PAYMENT,4,2,403,1,129.99,129.99
4,2013-07-25,8827,CLOSED,5,4,897,2,49.98,24.99
4,2013-07-25,8827,CLOSED,6,4,365,5,299.95,59.99
4,2013-07-25,8827,CLOSED,7,4,502,3,150.0,50.0
4,2013-07-25,8827,CLOSED,8,4,1014,4,199.92,49.98
5,2013-07-25,11318,COMPLETE,9,5,957,1,299.98,299.98
5,2013-07-25,11318,COMPLETE,10,5,365,5,299.95,59.99


In [0]:
display(order_details_df.select(orders_df['order_id'], 'order_Date', 'order_customer_id', 'order_item_subtotal').limit(10))

order_id,order_Date,order_customer_id,order_item_subtotal
1,2013-07-25,11599,299.98
2,2013-07-25,256,199.99
2,2013-07-25,256,250.0
2,2013-07-25,256,129.99
4,2013-07-25,8827,49.98
4,2013-07-25,8827,299.95
4,2013-07-25,8827,150.0
4,2013-07-25,8827,199.92
5,2013-07-25,11318,299.98
5,2013-07-25,11318,299.95


In [0]:
display(order_details_df.select(orders_df['*']).limit(10))

order_id,order_date,order_customer_id,order_status
1,2013-07-25,11599,CLOSED
2,2013-07-25,256,PENDING_PAYMENT
2,2013-07-25,256,PENDING_PAYMENT
2,2013-07-25,256,PENDING_PAYMENT
4,2013-07-25,8827,CLOSED
4,2013-07-25,8827,CLOSED
4,2013-07-25,8827,CLOSED
4,2013-07-25,8827,CLOSED
5,2013-07-25,11318,COMPLETE
5,2013-07-25,11318,COMPLETE


In [0]:
display(order_details_df.select(orders_df['*'], 'order_item_subtotal').limit(10))

order_id,order_date,order_customer_id,order_status,order_item_subtotal
1,2013-07-25,11599,CLOSED,299.98
2,2013-07-25,256,PENDING_PAYMENT,129.99
2,2013-07-25,256,PENDING_PAYMENT,250.0
2,2013-07-25,256,PENDING_PAYMENT,199.99
4,2013-07-25,8827,CLOSED,199.92
4,2013-07-25,8827,CLOSED,150.0
4,2013-07-25,8827,CLOSED,299.95
4,2013-07-25,8827,CLOSED,49.98
5,2013-07-25,11318,COMPLETE,129.99
5,2013-07-25,11318,COMPLETE,299.98


compute daily revenue for orders which are placed in 2014 january and alse the order status is in complete or close

In [0]:
from pyspark.sql.functions import year, month

In [0]:
display(order_details_df.filter("order_status in ('COMPLETE', 'CLOSED') and year(order_date) = 2014 and month(order_date) = 1"). \
    groupBy('order_date'). \
    agg(round(sum('order_item_subtotal'), 2).alias('revenue')). \
    orderBy(col('revenue').desc())
)

order_date,revenue
2014-01-05,59093.58
2014-01-11,58913.51
2014-01-30,58597.63
2014-01-21,56125.68
2014-01-03,53080.1
2014-01-10,52781.11
2014-01-15,50622.14
2014-01-22,49359.94
2014-01-20,43416.32
2014-01-16,42362.41


In [0]:
display(orders_df.join(order_items_df,  orders_df['order_id'] == order_items_df['order_item_order_id'], 'left'). \
    select(orders_df['*'], order_items_df['order_item_subtotal']). \
    orderBy('order_id').limit(10))

order_id,order_date,order_customer_id,order_status,order_item_subtotal
1,2013-07-25,11599,CLOSED,299.98
2,2013-07-25,256,PENDING_PAYMENT,129.99
2,2013-07-25,256,PENDING_PAYMENT,250.0
2,2013-07-25,256,PENDING_PAYMENT,199.99
3,2013-07-25,12111,COMPLETE,null
4,2013-07-25,8827,CLOSED,199.92
4,2013-07-25,8827,CLOSED,150.0
4,2013-07-25,8827,CLOSED,299.95
4,2013-07-25,8827,CLOSED,49.98
5,2013-07-25,11318,COMPLETE,129.99


In [0]:
display(orders_df.join(order_items_df,  orders_df['order_id'] == order_items_df['order_item_order_id']).count())

172198

In [0]:
display(orders_df.join(order_items_df,  orders_df['order_id'] == order_items_df['order_item_order_id'], 'left').count())

183650

In [0]:
display(order_items_df.filter("order_item_order_id = 26"))

order_id,order_item_order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price


In [0]:
display(orders_df.count())

68883

In [0]:
display(order_items_df.count())

172198